In [71]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from geopy.distance import distance
import math

# Falta saber si son poligonos en un plano o puntos en un globo 

### Prueba consulta de restaurantes 

In [6]:
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        result = session.execute(
            'select * from restaurants limit 10')

In [7]:
for item in result:
    print(item)

Row(id='55cba2476c522cafdb058fdf', location=[-73.8048161, 40.7092884], name='The Smile Of The Beyond', type='Point')
Row(id='55cba2476c522cafdb05821e', location=[-74.009073, 40.63628], name="Mr.Q'S Gourmet", type='Point')
Row(id='55cba2476c522cafdb054d62', location=[-74.00208219999999, 40.7076928], name="Jeremy'S Ale House", type='Point')
Row(id='55cba2476c522cafdb05470a', location=[-73.91466319999999, 40.7430799], name='Boston Market', type='Point')
Row(id='55cba2476c522cafdb056cd4', location=[-73.90678129999999, 40.8774338], name='Pick Up Six Fresh Fast Asian Kitchen', type='Point')
Row(id='55cba2476c522cafdb055856', location=[-73.98073339999999, 40.7557186], name='Freefoods', type='Point')
Row(id='55cba2476c522cafdb0574e5', location=[-73.9629278, 40.7791655], name='Metropolitan Museum Roof Top Garden Cafe', type='Point')
Row(id='55cba2476c522cafdb056814', location=[-73.9989528, 40.7140481], name='Lobster Boat Restaurant', type='Point')
Row(id='55cba2486c522cafdb0598a4', location=[-7

### Extraer un vecindario tipo MultiPolygono
* 55cb9c666c522cafdb053a97

In [19]:
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        result = session.execute(
            'select * from neighborhoods where id = \'55cb9c666c522cafdb053a97\'')
        length = session.execute(
            'select count(*) from neighborhoods where id = \'55cb9c666c522cafdb053a97\'')

In [20]:
print('Puntos extraidos: '+str(length[0][0]) + '\n\nImprime puntos')
for row in result:
    print(row)

Puntos extraidos: 271

Imprime puntos
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=0, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[-73.96230391870556, 40.73311366098218])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=1, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[-73.96226340489365, 40.7329155516237])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=2, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[-73.96236043096485, 40.732913300829765])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=3, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[-73.96285045581647, 40.73287557416924])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=4, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[-73.96332992178779, 40.73283805163722])
Row(id='55cb9c666c522cafdb053a97', polygonNumber=0, pointNumber=5, name='Murray Hill-Kips Bay', type='MultiPolygon', point=[-73.96334075230253

### Encontrar restaurantes en una distancia específica respecto a un punto concreto
* Distancia = 1km
* Punto = [-73.94378455587042, 40.6973697290538]

In [69]:
distancia = 1
puntoInicial = [-73.94378455587042, 40.6973697290538]

In [77]:
with Cluster() as cluster:
    with cluster.connect('challenge') as session:
        result = session.execute('select * from restaurants')
        filtrado = []
        for row in result:
            #if distance(puntoInicial, row[1]).km <= distancia:
            if row[1][0] > 90 or row[1][0] < -90 or row[1][1] > 180 or row[1][1] < -180:
                filtrado.append([row[0], row[1], row[2]])

In [78]:
print('Numero de restaurantes dentro del límite',
      len(filtrado), f'\n\nImprimir restaurantes a distancia {distancia} desde {puntoInicial}:')
for item in filtrado:
    print(item, '| Distancia:', math.dist(puntoInicial, item[1]))

Numero de restaurantes dentro del límite 123 

Imprimir restaurantes a distancia 1 desde [-73.94378455587042, 40.6973697290538]:
['55cba2476c522cafdb054655', [-92.72249769999999, 41.7461483], "Sammy'S Noodle Shop & Grill"] | Distancia: 18.807977133131054
['55cba2476c522cafdb054035', [-90.1910491, 38.6144575], 'City College Cafeteria'] | Distancia: 16.38023591163668
['55cba2476c522cafdb058856', [-92.72786699999999, 41.7461428], 'Deli & Grocery'] | Distancia: 18.81333777462677
['55cba2476c522cafdb055a1d', [-104.2081266, 39.6086401], 'Rose House'] | Distancia: 30.28391869572445
['55cba2476c522cafdb055aa3', [-95.3356714, 37.4710885], 'Moca Asian Bistro'] | Distancia: 21.633809496272004
['55cba2476c522cafdb054cae', [-92.72296209999999, 41.746136], "Mcdonald'S"] | Distancia: 18.808440124715677
['55cba2476c522cafdb056f94', [-92.72874879999999, 41.7461464], 'Flavaboom'] | Distancia: 18.814218404141783
['55cba2486c522cafdb0596d2', [-92.7293043, 41.7461747], 'Soho Fried Chicken & Pizzeria'] | Di